In [5]:
# import packages
import pandas as pd
import numpy as np

In [2]:
# import data
X_train = pd.read_csv("Project_materials/ais_train.csv", delimiter='|')
X_test = pd.read_csv("Project_materials/ais_train.csv", delimiter='|')
X_train.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3


In [3]:
df=X_train.copy()
# 2. sog
# many of these have value 0, like half of the observations, while very few (only 405) have value above 100
# none have value 1023 -> no missing values!
print(df[df.sog>=100].sog.count())
df.shape
#df.head()

405


(1522065, 11)

In [4]:
# 4. heading
print(df[df.heading==511].heading.count())
# df[df.heading=511].heading didn't work, creates a copy of the dataframe, not the dataframe itself
# 4895 missing values
df.loc[df.heading==511, "heading"]=np.nan
print(df[df.heading==511].heading.count())

4895
0


In [5]:
# 6. etaraw
df.etaRaw.head()
# dates and times, won't touch it for now

0    01-09 23:00
1    12-29 20:00
2    01-02 09:00
3    12-31 20:00
4    01-25 12:00
Name: etaRaw, dtype: object

In [6]:
# 8. longitude
# no missing values
df[df.longitude==181].longitude.count()

0

In [13]:
# 10 portId
# 1615 missing values values, not sure why, not even sure what form they have
print(df.shape[0]-df.portId.count())
# 772 unique values, unique ports
print(df.portId.nunique())
null_values=df[df["portId"].isnull()]
print(null_values)
#it's good that I found the values at least lol

1615
772
                        time    cog   sog  rot  heading  navstat       etaRaw  \
33       2024-01-01 00:13:51  257.0   0.0    0    257.0        5  12-31 05:00   
195      2024-01-01 00:35:00  257.0   0.0    0    257.0        5  12-31 05:00   
359      2024-01-01 00:56:08  257.0   0.0    0    257.0        5  12-31 05:00   
523      2024-01-01 01:17:20  256.0   0.0    0    256.0        5  12-31 05:00   
608      2024-01-01 01:35:27  256.0   0.0    0    256.0        5  12-31 05:00   
...                      ...    ...   ...  ...      ...      ...          ...   
1279313  2024-04-18 00:36:16  145.6   0.0   -5     18.0        5  04-17 19:00   
1279505  2024-04-18 00:57:15  265.0   0.0   -5     18.0        5  04-17 19:00   
1279611  2024-04-18 01:15:16  200.2   0.0    0     18.0        5  04-17 19:00   
1429638  2024-04-30 06:56:35  117.6   7.6  127    119.0        0  04-29 19:50   
1429829  2024-04-30 07:17:55  175.3  12.4  127    178.0        0  04-29 19:50   

         latitude 

In [7]:
ports=pd.read_csv("Project_materials/ports.csv", delimiter='|')
df=ports.copy()

In [11]:
df.head()


,portId,name,portLocation,longitude,latitude,UN_LOCODE,countryName,ISO
0,61d36ed80a1807568ff9a064,Port of Algiers,Algiers,3.067222,36.773611,DZALG,Algeria,DZ
1,61d36ed80a1807568ff9a065,Port of Annaba,Annaba,7.772500,36.900556,DZAAE,Algeria,DZ
2,61d36edf0a1807568ff9a070,Port of Oran,Oran,-0.639722,35.712222,DZORN,Algeria,DZ
3,61d36ee00a1807568ff9a072,Port of Skikda,Skikda,6.905833,36.887500,DZSKI,Algeria,DZ
4,61d36ee10a1807568ff9a074,Port of Pago-Pago,Pago-Pago,-170.690556,-14.274167,ASPPG,American Samoa,AS
